<a href="https://colab.research.google.com/github/abinashgochhayat1998/BasicMachineLearningAlgorithms/blob/main/M200674CA_Abinash_NaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import all the required packages
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns

##load our data set from computer 
import io
from google.colab import files
uploaded = files.upload()

Saving dermatology.csv to dermatology.csv


In [ ]:
dataframe = pd.read_csv(io.BytesIO(uploaded['dermatology.csv']))
dataframe.head(20) 

,erythema,scaling,definite_borders,itching,koebner_phenomenon,polygonal_papules,follicular_papules,oral_mucosal_involvement,knee_and_elbow_involvement,scalp_involvement,family_history,melanin_incontinence,eosinophils_in_the_infiltrate,pnl_infiltrate,fibrosis_of_the_papillary_dermis,exocytosis,acanthosis,hyperkeratosis,parakeratosis,clubbing_of_the_rete_ridges,elongation_of_the_rete_ridges,thinning_of_the_suprapapillary_epidermis,spongiform_pustule,munro_microabcess,focal_hypergranulosis,disappearance_of_the_granular_layer,vacuolisation_and_damage_of_basal_layer,spongiosis,saw-tooth_appearance_of_retes,follicular_horn_plug,perifollicular_parakeratosis,inflammatory_monoluclear_inflitrate,band-like_infiltrate,age,class
0,2,2,0,3,0,0,0,0,1,0,0,0,0,0,0,3,2,0,0,0,0,0,0,0,0,0,0,3,0,0,0,1,0,55.0,2
1,3,3,3,2,1,0,0,0,1,1,1,0,0,1,0,1,2,0,2,2,2,2,2,1,0,0,0,0,0,0,0,1,0,8.0,1
2,2,1,2,3,1,3,0,3,0,0,0,1,0,0,0,1,2,0,2,0,0,0,0,0,2,0,2,3,2,0,0,2,3,26.0,3
3,2,2,2,0,0,0,0,0,3,2,0,0,0,3,0,0,2,0,3,2,2,2,2,0,0,3,0,0,0,0,0,3,0,40.0,1
4,2,3,2,2,2,2,0,2,0,0,0,1,0,0,0,1,2,0,0,0,0,0,0,0,2,2,3,2,3,0,0,2,3,45.0,3
5,2,3,2,0,0,0,0,0,0,0,0,0,2,1,0,2,2,0,2,0,0,0,1,0,0,0,0,2,0,0,0,1,0,41.0,2
6,2,1,0,2,0,0,0,0,0,0,0,0,0,0,3,1,3,0,0,0,2,0,0,0,0,0,0,0,0,0,0,2,0,18.0,5
7,2,2,3,3,3,3,0,2,0,0,0,2,0,0,0,2,3,0,0,0,0,0,0,0,0,2,2,3,2,0,0,3,3,57.0,3
8,2,2,1,0,2,0,0,0,0,0,0,0,0,0,0,2,1,0,1,0,0,0,0,0,0,0,0,2,0,0,0,2,0,22.0,4
9,2,2,1,0,1,0,0,0,0,0,0,0,0,0,0,3,2,0,2,0,0,0,0,0,0,0,0,2,0,0,0,2,0,30.0,4


In [ ]:
dataframe.shape

(366, 35)

In [ ]:
dataframe.isnull().sum()#in our data set we have any missing values so we will drop the age column because it is not a categorical data

erythema                                    0
scaling                                     0
definite_borders                            0
itching                                     0
koebner_phenomenon                          0
polygonal_papules                           0
follicular_papules                          0
oral_mucosal_involvement                    0
knee_and_elbow_involvement                  0
scalp_involvement                           0
family_history                              0
melanin_incontinence                        0
eosinophils_in_the_infiltrate               0
pnl_infiltrate                              0
fibrosis_of_the_papillary_dermis            0
exocytosis                                  0
acanthosis                                  0
hyperkeratosis                              0
parakeratosis                               0
clubbing_of_the_rete_ridges                 0
elongation_of_the_rete_ridges               0
thinning_of_the_suprapapillary_epi

In [ ]:
dataframe.drop('age', axis = 'columns', inplace = True)
dataframe.head()

,erythema,scaling,definite_borders,itching,koebner_phenomenon,polygonal_papules,follicular_papules,oral_mucosal_involvement,knee_and_elbow_involvement,scalp_involvement,family_history,melanin_incontinence,eosinophils_in_the_infiltrate,pnl_infiltrate,fibrosis_of_the_papillary_dermis,exocytosis,acanthosis,hyperkeratosis,parakeratosis,clubbing_of_the_rete_ridges,elongation_of_the_rete_ridges,thinning_of_the_suprapapillary_epidermis,spongiform_pustule,munro_microabcess,focal_hypergranulosis,disappearance_of_the_granular_layer,vacuolisation_and_damage_of_basal_layer,spongiosis,saw-tooth_appearance_of_retes,follicular_horn_plug,perifollicular_parakeratosis,inflammatory_monoluclear_inflitrate,band-like_infiltrate,class
0,2,2,0,3,0,0,0,0,1,0,0,0,0,0,0,3,2,0,0,0,0,0,0,0,0,0,0,3,0,0,0,1,0,2
1,3,3,3,2,1,0,0,0,1,1,1,0,0,1,0,1,2,0,2,2,2,2,2,1,0,0,0,0,0,0,0,1,0,1
2,2,1,2,3,1,3,0,3,0,0,0,1,0,0,0,1,2,0,2,0,0,0,0,0,2,0,2,3,2,0,0,2,3,3
3,2,2,2,0,0,0,0,0,3,2,0,0,0,3,0,0,2,0,3,2,2,2,2,0,0,3,0,0,0,0,0,3,0,1
4,2,3,2,2,2,2,0,2,0,0,0,1,0,0,0,1,2,0,0,0,0,0,0,0,2,2,3,2,3,0,0,2,3,3


In [ ]:
dataframe=dataframe.to_numpy()#converting our dataframe to numpy arrays inorder to perform better calculations

In [ ]:
def train_naiveBayesModel(X, y):
    classes = np.unique(y)
    n_classes = len(classes)
    numberOfSamples, numberOfFeatures = X.shape

    # calculate mean, var, and prior for each class
    mean = np.zeros((n_classes, numberOfFeatures), dtype=np.float64)
    variance = np.zeros((n_classes, numberOfFeatures), dtype=np.float64)
    prior_probability = np.zeros(n_classes, dtype=np.float64)

    for idx, c in enumerate(classes):
        X_c = X[y == c]
        mean[idx, :] = X_c.mean(axis=0)+0.1
        variance[idx, :] = X_c.var(axis=0)+0.1
        prior_probability[idx] = X_c.shape[0] / float(numberOfSamples)
    return mean, variance, prior_probability, classes

In [ ]:
def classConditionalProbability(x,classes,prior_prob,mean,variance):
    posteriors = []

    # calculate posterior probability for each class
    for index, c in enumerate(classes):
        prior = np.log(prior_prob[index])
        posterior = np.sum(np.log(gaussian(index, x,mean,variance)))
        posterior = prior + posterior
        posteriors.append(posterior)

    # return class with highest posterior probability
    return classes[np.argmax(posteriors)]

In [ ]:
def predict(X,classes,prior,mean,variance):
    y_pred = [classConditionalProbability(x, classes, prior, mean, variance) for x in X]
    return np.array(y_pred)

In [ ]:
def gaussian(idx, x,mean,variance):
    mean = mean[idx]
    variance = variance[idx]
    numerator = np.exp(-((x - mean) ** 2) / (2 * variance))
    denominator = np.sqrt(2 * np.pi * variance)
    return numerator / denominator

In [ ]:
def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

In [ ]:
from sklearn.model_selection import train_test_split
x_independent_variable=dataframe[:,:-1]
y_dependent_variable=dataframe[:,-1]
X_train, X_test, y_train, y_test = train_test_split( x_independent_variable, y_dependent_variable, train_size=0.70)

print(X_train.shape)
print(y_train.shape)

(256, 33)
(256,)


In [ ]:
mean, variance, prior, classes = train_naiveBayesModel(X_train, y_train)
predictions = predict(X_test, classes, prior, mean, variance)
    
print(f'Naive Bayes Model classification accuracy={"{:.2f}".format(accuracy(y_test, predictions)*100)}%')

Naive Bayes Model classification accuracy=98.18%


In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00        34
           2       1.00      0.87      0.93        15
           3       1.00      1.00      1.00        22
           4       0.88      1.00      0.93        14
           5       1.00      1.00      1.00        16
           6       1.00      1.00      1.00         9

    accuracy                           0.98       110
   macro avg       0.98      0.98      0.98       110
weighted avg       0.98      0.98      0.98       110

